FACE DETECTION SYSTEM

importing the libraries

In [210]:
import os

In [211]:
import pandas as pd

In [212]:
import numpy as np

In [213]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [214]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [215]:
from tensorflow.keras.preprocessing import image

In [216]:
from keras.preprocessing import image

In [217]:
import cv2

In [218]:
categories=['without_mask','withmask']

In [219]:
data=[]
for category in categories:
    path=os.path.join('train',category)

    label=categories.index(category)

    for file in os.listdir(path):

        img_path=os.path.join(path,file)
        img=cv2.imread(img_path)
        img=cv2.resize(img,(224,224))

        data.append([img,label])
        
        

In [220]:
len(data)

1024

In [221]:
import random

In [222]:
random.shuffle(data)

In [223]:
X=[]
y=[]

for features,label in data:
    X.append(features)
    y.append(label)

In [224]:
X=np.array(X)
y=np.array(y)

In [225]:
X.shape

(1024, 224, 224, 3)

In [226]:
y.shape

(1024,)

In [227]:
X=X/255

In [228]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [229]:
X_test.shape

(205, 224, 224, 3)

In [230]:
y_test.shape

(205,)

In [231]:
from keras.applications.vgg16 import VGG16

In [232]:
vgg=VGG16()

In [233]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [234]:
from keras import Sequential

In [235]:
model=Sequential()

In [236]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [237]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [238]:
for layer in model.layers:
    layer.trainable=False

In [239]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [240]:
from keras.layers import Dense

In [241]:
model.add(Dense(1,activation='sigmoid'))

In [242]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [243]:
from tensorflow.keras.optimizers.legacy import Adam

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [244]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_set = train_datagen.flow_from_directory(
    'train',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'   # 👈 add this
)


Found 1024 images belonging to 2 classes.


In [245]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Training data generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 80% training, 20% validation
)

train = train_datagen.flow_from_directory(
    "train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val = train_datagen.flow_from_directory(
    "train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 820 images belonging to 2 classes.
Found 204 images belonging to 2 classes.


In [246]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False


In [247]:
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)


In [248]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [249]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [263]:
model.fit(train_set,
          validation_data=train_set,
          epochs=10,
          callbacks=[early_stop])


Epoch 1/10
32/32 [==============================] - 29s 913ms/step - loss: 0.0058 - accuracy: 0.9990 - val_loss: 0.0045 - val_accuracy: 0.9990
Epoch 2/10
32/32 [==============================] - 28s 884ms/step - loss: 0.0194 - accuracy: 0.9961 - val_loss: 0.0020 - val_accuracy: 0.9990
Epoch 3/10
32/32 [==============================] - 28s 883ms/step - loss: 0.0046 - accuracy: 0.9980 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 4/10
32/32 [==============================] - 28s 882ms/step - loss: 0.0126 - accuracy: 0.9951 - val_loss: 2.8701e-04 - val_accuracy: 1.0000
Epoch 5/10
32/32 [==============================] - 28s 877ms/step - loss: 0.0136 - accuracy: 0.9951 - val_loss: 0.0201 - val_accuracy: 0.9951
Epoch 6/10
32/32 [==============================] - 28s 876ms/step - loss: 0.0077 - accuracy: 0.9980 - val_loss: 0.0067 - val_accuracy: 0.9990
Epoch 7/10
32/32 [==============================] - 28s 877ms/step - loss: 0.0113 - accuracy: 0.9971 - val_loss: 0.0075 - val_accuracy: 0.

In [252]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

for (x, y, w, h) in faces:
    face = frame[y:y+h, x:x+w]
    face_resized = cv2.resize(face, (224,224))
    y_pred = detect_face_mask(face_resized)


1/1 [==============================] - 0s 43ms/step
Raw prediction: 0.99636585


In [253]:
import cv2

def draw_label(img, text, pos, bg_color):
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, cv2.FILLED)
    end_x = pos[0] + text_size[0][0] + 2
    end_y = pos[1] + text_size[0][1] - 2

    cv2.rectangle(img, pos, (end_x, end_y), bg_color, cv2.FILLED)
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1, cv2.LINE_AA)


In [254]:
y_pred = detect_face_mask(face)

# interpret prediction correctly
if y_pred == 0:
    label = "Mask"
    color = (0, 255, 0)     # green for mask
else:
    label = "No Mask"
    color = (0, 0, 255)     # red for no mask

# draw box + label
cv2.putText(frame, label, (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)


1/1 [==============================] - 0s 45ms/step
Raw prediction: 0.99636585


array([[[ 35,  35,  35],
        [ 34,  34,  34],
        [ 32,  32,  32],
        ...,
        [135, 139, 128],
        [134, 138, 127],
        [133, 137, 126]],

       [[ 34,  34,  34],
        [ 33,  33,  33],
        [ 31,  31,  31],
        ...,
        [135, 139, 128],
        [134, 138, 127],
        [133, 137, 126]],

       [[ 30,  30,  30],
        [ 29,  29,  29],
        [ 28,  28,  28],
        ...,
        [135, 139, 128],
        [134, 138, 127],
        [133, 137, 126]],

       ...,

       [[ 83,  83,  83],
        [ 83,  83,  83],
        [ 83,  83,  83],
        ...,
        [184, 148, 118],
        [208, 171, 140],
        [236, 199, 166]],

       [[ 83,  83,  83],
        [ 83,  83,  83],
        [ 83,  83,  83],
        ...,
        [185, 149, 119],
        [196, 159, 128],
        [218, 181, 148]],

       [[ 83,  83,  83],
        [ 83,  83,  83],
        [ 83,  83,  83],
        ...,
        [191, 155, 125],
        [194, 157, 126],
        [212, 175, 142]]

In [264]:
import cv2
import numpy as np

# ---------- Load Model ----------
# Make sure 'model' is already loaded before running this cell
# Example: model = load_model('your_model_name.h5')

# ---------- Load Face Detector ----------
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# ---------- Start Video Capture ----------
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # Loop through detected faces
    for (x, y, w, h) in faces:
        face = frame[y:y + h, x:x + w]
        face = cv2.resize(face, (224, 224))  # Resize for model
        face = face / 255.0                  # Normalize
        face = np.expand_dims(face, axis=0)  # Add batch dimension

        # Prediction
        y_pred = model.predict(face)[0][0]

        # ---------- Display Result ----------
        if y_pred > 0.2:
            label = "No Mask"
            color = (0, 0, 255)
            # Red for no mask
        elif y_pred < 0.7:
            label = "Mask"
            color = (0, 255, 0)
        else:
            print('not detectable')
        
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, f"{label} ({y_pred:.2f})", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, cv2.LINE_AA)

    # ---------- Show Video ----------
    cv2.imshow("Mask Detection", frame)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ---------- Cleanup ----------
cap.release()
cv2.destroyAllWindows()


In [255]:
print("Raw prediction:", y_pred)

Raw prediction: No Mask


In [256]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_4[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']      

In [257]:
import cv2
import numpy as np

img = cv2.imread("wom.jpg")
img = cv2.resize(img, (224,224)) / 255.0
pred = model.predict(img.reshape(1,224,224,3))[0][0]
print("Raw prediction:", pred)


1/1 [==============================] - 0s 46ms/step
Raw prediction: 0.99999857


In [258]:
mg = cv2.imread("wm.jpg")
img = cv2.resize(img, (224,224)) / 255.0
pred = model.predict(img.reshape(1,224,224,3))[0][0]
print("Raw prediction:", pred)


1/1 [==============================] - 0s 43ms/step
Raw prediction: 0.9712799
